![figure](../lab1/lab1_figures/politecnico_h-01.png)
# **Eletrónica Configurável / Configurable Electronics**
#### Mestrado em Engenharia Eletrotécnica / Master in Electrical and Electronic Engineering

## **LabWork5 - Multirate Systems**

____________

## Introduction ##
In this lab you will explore multirate filter implementation using the Polyphase Interpolator option in the FIR compiler IP core from Xilinx. For data generation you will resort to a simple shift register. Results will be observed using an Integrated Logic Analyser. (ILA).



### Objectives ###
After completing this lab, you will be able to:

* Initialize and configure a shift register IP core
* Initialize and configure Polyphase filters and its AXI interface
* Understand the concept of multirate signal processing 
* Build a digital up-converter and evaluate its performance in hardware.


In the instructions below **{sources}** refers to `C:\Xilinx\MEE_EC\sources` and **{labs}** refers to `(C:\Xilinx\MEE_EC\labs)`


### Design description ###

The block diagram shown in the following figure shows the high level structure of the circuit we will design. It includes 

____________

## Step 1 - Create a Vivado Design ##

### Step 1.1 ###

In this section you will create a new and empty project for PYNQ-Z2 Board in Vivado. We will use Vivado 2020.1 - note that some options may vary depending on the version you are using.

* Start downloading the source files provided (Moodle) and save them in `{sources}/lab5`.


* Open Vivado and create a new project to start the wizard. Click **Next**.


* Click the browse button of the project location field of the new project form, browse to `c:/Xilinx/MEE_EC/labs` and click **Select**. Enter `lab5` in the project name field. Make sure that the *Create Project Subdirectory* box is checked. Click **Next**.


* We will create an empty RTL project and specify sources later. Select **RTL Project** option in the *Project Type* form, and click **Next**. Make sure that the *Do not specify sources* at this time box is checked.


* In the *Default Part* form, select the *Boards* option and choose the **PYNQ-Z2** board. Click **Next** and then **Finish** to create the Vivado project.


* In the *Sources* pane select **Add Sources** by clicking the **plus** sign. Then select *Add or create constraints* and click **Next**.


* Select *Add Files* and browse to the `{sources}\lab5` directory, select `lab5.xdc`, click **OK** and then **Finish**. 


### Step 1.2 ### 

In this section you will create the clock synthesis module.


* In the *Flow Navigator*, expand *IP INTEGRATOR* and select **Create Block Design**. Name it **top_lab5** anc click **OK**. 


* Add the following Xilinx IP cores to the design canvas:
    * 1 Clocking Wizard
    * 1 Utility Buffer
    * 1 Slice
    * 1 Binary Counter
    
      
* Double click the **Clocking Wizard** to configure it. The input clock should be set to **125MHz**, from which two other clocks should be generated at a frequency of **100MHZ** and **32MHZ**. 

 
<div class="alert alert-block alert-info">
<b>Info:</b> This 32MHz frequency will be used to generate two low frequency clocks (**1MHz** and **125KHz**) using one 8-bit counter, two slicers and two global buffers.
</div>


* Double-click the **Binary Counter** and select an **Output Width = 8** and one **Clock Enable (CE)** input.


* Double-click the **Utility Buffers** and select **BUFG** in bth.


* Double-click one **Slice** and select an **Din Width = 8** from 7 downto 7 (Dout Width=1). This will be used to output a 125kHz clock. The other should be **Din Width = 8** from 4 downto 4 (Dout Width=1), and outputs a 1MHz.


* Make the connections, create the external ports and change the names according to the figure. Make sure you have no errors when you finish (select **Validate Design**) and **Regenerate Layout** to have a clear and tiddy block design.

![Figure](../lab5/lab5_figures/fig2.png)


* Now select all the modules simultaneously, **righ-click** on them and select **Create Hierarchy** from the drop-down menu. Call it **Clock_Synth**. This creates an Hierarchical module with these internals, which is responsible for generating and the clocks needed in this design.
 
 
* Delete the output ports of this **clk_synth** module.

### Step 1.3 ### 

In this step you will create a binary signal generation, to be read at the rate of 125kbaud (symbols/sec), and generate raised cosine pulses at a much higher sampling rate (32MHz).
     
    
* In the top level design canvas, add the following Xilinx IP cores:
    * 1 RAM-based Shift Register
    * 1 FIR Compiler
    * 1 Integrated Logic Analyzer (ILA)
  

* We will now initializate a shift register with 16 symbols with amplitude defined by 16 bits. Some have amplitude 7FFF and others have amplitude 0000. The maximum valu was set to hexadecimal 7FFF because data input to FIR is always considered to be **signed** (two's complement). Double click one of the **RAM-based Shift Register**, name it **data_gen**, and set the following:
    * Configuration tab -> Shift register Type: Fixed Length
    * Configuration tab -> Dimensions: Width=16; Depth=16;
    * Initialization tab -> COE File : use the **mem.coe** file given in `{sources}\lab5`


* We will now create a rectangular interpolation filter with 8 coeficients equal to 1/8 (no normalize energy to 1). The output should be up-sampled 8x, i.e, the output sample rate will be 1MHz. Double click the **FIR Compiler** and set the following:
    * Filter Options tab -> Filter Coefficients -> COE File : use the **rect.coe** file given in `{sources}\lab5`
    * Filter Options tab -> Filter Specification -> Filter Type : Interpolation
    * Filter Options tab -> Filter Specification -> Interpolation Rate Value : 8
    * Channel Specification tab -> Hardware Oversampling Specifications -> Select Format : Frequency Specification
    * Channel Specification tab -> Hardware Oversampling Specifications -> Input Sampling Frequency : 0.125
    * Channel Specification tab -> Hardware Oversampling Specifications -> Clock Frequency : 1
    * Implementation tab -> Coefficient Options -> Quantization : Quantize Only (16 bit width and 14 fractional)
    * Implementation tab -> Data Path Options -> Output rounding Mode : Truncate LSBs; 16 output width


* Double click the **ILA** and set the following:
    * General Options tab -> Monitor Type: Native
    * General Options tab -> Number of Probes : 4
    * General Options tab -> Trigger and Storage Settings -> Select Capture and Advanced Trigger
    * Probe Ports tab -> Probe0: 16 bits; Probe1: 16bits; Probe2:8bits; Probe3:1bits
    

* Double click the other **RAM-based Shift Register**, name it **tvalid_gen**. This module will generate a control signal indicating when there is a new data sample available for the filter. Because the filter is clocked at 1MHz, the tvalid pulse  should have a **time high = 1us** and period **8us**. Double-click it and set the following:
    * Configuration tab -> Shift register Type: Fixed Length
    * Configuration tab -> Dimensions: Width=1; Depth=8;
    * Initialization tab -> COE File : use the **tvalid.coe** file given in `{sources}\lab5`


* Complete the connections, validate and regenerate the design until you have the following:

![Figure](../lab5/lab5_figures/fig3.png)


* **Save** the diagram and, in the *Sources* window, **righ-click** the **top_lab5** block design and select **Create HDL Wrapper** letting Vivado manage it.


* Select **Create Bitstream** to test the design in hardware. Vivado will synthesize, implement the design and ask you what next step you want to select. Choose **Open the Hardware Manager** and click **ok**.


* Turn-on the board and programm it. We don't want to capture all samples at the ILA clock rate (100MHz) so we need to configure its capture mode. Set the **Capture Mode** to **Basic** and in the **Capture setup** window add the counter. Set the capture frequency to 1MHz, i.e., the capture must happen every time the counter's fourth bit rises (== XXXR_XXXX). 


* Configure the filter's input and output signals' *Waveform Style* as **Analog** and Radix **Signed Decimal**, in order to see the data. You should see that the output data matches the input data after some latency (3 data periods). This is what we should expect from a rectangular polyphase filter.



![Figure](../lab5/lab5_figures/fig4.png)


* **Close** the *Hardware Manager*.

______

## Step 2 - Implement a Pulse Shapping Filter ##

### Step 2.1 ### 

On this step you will implement the a pulse shapping filter in order to obtain root raised cosine pulses at the filter's output.
     
    
* Open **Matlab** and run the **fdatool**. Configure a **square root raised cosine** filter with **32 taps**, 1MHz sampling frequency, 62.5kHz cut-off frequency and a **roll-off** equal to 0.5, as shown bellow.


![Figure](../lab5/lab5_figures/fig5.png)


<div class="alert alert-block alert-info">
<b>Info:</b> Note that the filter's coefficients show it's rasised cosine impulse response, with 8 samples between the maximum and the first null, i.e, with 8us beween the maximum and the first null. Note that the filter is working at 1MHz, so the time between output samples will be 1us. This means we will have no ISI between data symbols.
</div>


* Quantize the filter to have **Fixed-point** arithmetic, **Full Precision** with 16-bit numerator and scalled to fully utilize the entire dynamic range, as shown bellow.

![Figure](../lab5/lab5_figures/fig6.png)


* **Export the coefficents** and save them in a file typing the following instruction in Matlab's Command Window: `csvwrite('rcosine.dat', Num);` Note that the data in this file is the same as the data stored in Matlab variable **Num**. Also note that these are the same values as the ones provided in `{sources}\lab5\rcosine.coe`.


* Open the **Block Design**, double-click the **FIR Compiler** and set the filter coefficients to **rcosine.coe**. Click **OK**.


* **Save** the block design and **Generate Bitstream** again. We expect to observe raised cosine pulses in the filter's output, with a resolution of 8 samples per symbol time. Check if this is true by reprogramming the FPGA and running the ILA with the same capture settings used before. results should be the same as shown bellow.


![Figure](../lab5/lab5_figures/fig7.png)


* **Close** the *Hardware Manager* and open **Project Summary**. 


* Look for the Post-Implementation **Utilization** table and observe that the 32 tap filter is using only 4 DSP modules, i.e., it is using only 4 multipliers. Can you explain why?


<div class="alert alert-block alert-info">
<b>Info:</b> Note that the N=32 prototype filter coefficients ($a_{0}, a_{1}, ..., a_{31}$) are mapped to M=8 polyphase subfilters, so each subfilter will have only p=32/8=4 coefficients. Because these subfilters are used in sequence (time-multiplexed), the hardware footprint is just 4 DSPs instead of 8x4=32 DSPs!
</div>

* Also note that the design is failling timing with a **Negative Slacks**. However, if you inspect the schematic location of these paths, you will see that these paths occur between the filter and the ILA, which run at different clock frequencies. This can be easily fixed by running the ILA at 32MHz to match the filter's ouput frequency!

### Step 2.2 ### 

In the last step you used a multi-rate interpolation filter to generate raised-cosine shapped pulses with fs=1MHz, from a 125kbaud data stream (8x rate change). Now, we will increase that sample rate ratio a little further to 256x, in order to be able to multiply these pulses by a 1MHz sine wave, generated with a sample rate of 32MHz. 
     
    
* Open **Matlab** and run the **fdatool**. Configure a **square root raised cosine** filter with **1024 taps**, 32MHz sampling frequency, 62.5kHz cut-off frequency and a **roll-off** equal to 0.5, as shown bellow.This filter should have 256 samples between the maximum and the first null.


![Figure](../lab5/lab5_figures/fig8.png)


* **Export** the coefficients and **save** them in a **rcosine2.coe** file.

<div class="alert alert-block alert-warning">
<b>Warning:</b> Check the file for unsupported data representation. If coefficients are represented with decimal floating point (e.g., 3e-5) you must convert that to normal decimal fixed point representation (0.00003).
</div>

* Open the block design and double-click the **FIR Compiler**. Set the coefficient file to **rcosine2.coe**, the interpolation rate to 256 and the input Clock frequency to **32MHz**. Click **Ok**.


* In the block design, disconnect the filter's clock pin (**aclk**) and connect it to the **32MHz clock**. Do the same for the **tvalid_gen** clock, which must now be connected to 32MHz.


* Double-click the **tvalid_gen** and increase the depth to **256**. Also change the initialization file to a deeper one, as defined in **tvalid2_coe**.


* **Validate the design** and if it passes, **regenerate the layout**. You should have the design as follows:

![Figure](../lab5/lab5_figures/fig9.png)


* **Save** the block design and **Generate Bitstream** again. If you have errors you may need to force the design to update and run output product again. To do that, go to the **Design Runs** tab, open the **Out-of-Context Module Runs**, right-click on **top_lab5** and select **Regenerate Output Products** and click **Reset** and then **Generate**.


* **Run the generate bitstream** process again. We expect to observe raised cosine pulses in the filter's output, but with a much finner resolution now, as the output sample frequency is now equal to 32MHz. Check if this is true by **reprogramming the FPGA** and running the ILA with the capture setup equal to XXX_XXXB (data is captured at 32MHz).


* Note that with these settings, the ILA data window depth (1024 samples) can only capture 4 symbol periods, which is low (see the left figure). To see more symbols we can downsample the ILA aquisition, by setting the capture to XXX_B. This decreases the  capturing rate to 4MHz and we can now observe 32 symbol periods (shown in the right figure).


![Figure](../lab5/lab5_figures/fig10.png)


* **Close** the *Hardware Manager* and open **Project Summary**. 


* Look for the Post-Implementation **Utilization** table and observe that the 1024 tap filter is again using only 4 DSP modules, i.e., it is using only 4 multipliers! Can you explain why?




_________

## Step 3 - Convert to passband ##

### Step 3.1 ###

In this step you will up-convert the baseband data signal by multiplying it with a 0.5MHz sinusoidal carrier, generated with the same 32MHz sample rate. 


* Open the **Block Design** and Add two new IP cores:
    * 1 DDS Compiler
    * 1 Multiplier
    

* Double click the **DDS** and configure it according to the following:
    * Configuration tab -> System Requirements -> System Clock (MHz): 32
    * Configuration tab -> System Requirements -> Parameter Selection: Hardware Parameters
    * Configuration tab -> Hardware Parameters -> Phase Width: 6 ; Output Width: 16
    * Implementation tab -> Output -> Output Selection: Sine
    * Implementation tab -> Output -> Has Phase Out: Uncheck
    * Phase angle Increment Values tab -> Value: 00001
    
    
* Double click the **Multiplier** and configure it according to the following:
    * Basic tab -> Input Options -> Width: 16 (in both A and B)
    * Output and Control tab -> Use Custom Output Width: Check ; Output MSB = 31; Output LSB = 16
    * Output and Control tab -> Pipelining and Control Signals -> Pipeline Stages: 4
 
 
* **Add two more probes** in ILA and connect them to the DDS output and to the multiplier output. The block design should look as shown below. 


![Figure](../lab5/lab5_figures/fig11.png)


* Validate the design and run **Generate Bitstream**.


* Add the two new probe signals to the waveform window and set them to **Analog** and **Signed**. 


* Configure the *Capture Setup* to capture data when **`clock_synth_Counter[7:0] == XXXX_BXXX`**, i.e., at a 4MHz frequency. With this should configuration you can observe a significant portion of the modulated signal but with a coarser resolution.


* **Zoom to Fit** and you should now be able to see the 0.5MHZ sine wave and the ASK modulated pulses, as shown bellow.  


![Figure](../lab5/lab5_figures/fig12.png)


* **Zoom In** and note that you are observing only 8 samples per sinusoid period because you are capturing the waveforms with a 4MHz frequency (sinusoid is 0.5MHz). 


* If you change the capturing frequency, e.g, **`clock_synth_Counter[7:0] == XXXX_XXBX`** you will see 32 samples per sinusoid period but fewer baseband pulses in the captured window. 


* Change the capture back to 4MHz (**`clock_synth_Counter[7:0] == XXXX_BXXX`**).




### Step 3.2 ###

In this step you will save the data obtained by the ILA core and observe the spectrum of baseband and passband signals.

* Go to the **Tcl Console** an run the following lines of code to save data in a file named **data1.csv**:
```
file mkdir C:/Xilinx/MEE_EC/labs/lab5_ILAcapture
run_hw_ila hw_ila_1
write_hw_ila_data -force -csv_file C:/Xilinx/MEE_EC/labs/lab5_ILAcapture/data1 [upload_hw_ila_data hw_ila_1]
```

![Figure](../lab5/lab5_figures/fig13.png)


* Open the file with **Notepad++** and note that it is a **Comma Separated Values** file with 9 collumns with the following organization:
    * Column 1: Sample in Buffer
    * Column 2: Sample in Window
    * Column 3: TRIGGER
    * Column 4: c_shift_ram_0_Q[15:0] -> rectangular pulses
    * Column 5: fir_compiler_0_m_axis_data_tdata[15:0] -> raised cosine pulses
    * Column 6: clock_synth_Counter[7:0]
    * Column 7: c_shift_ram_1_Q -> start pulse
    * Column 8: dds_compiler_0_m_axis_data_tdata[15:0] -> 0.5MHz sine wave
    * Column 9: mult_gen_0_P[15:0] -> passband ask signal
   

* Now you have to open this file in **Matlab** to extract the data from **columns 4, 5, and 9**, i.e., samples belonging to the baseband and passband streams. To do that, run the m-file **VivadoILA_OpenExported.m** provided in {sources}/lab5.


* **Plot** the final extracted data streams and check if data is as expected:
```
subplot(3,1,1), stem(bin_vec_baseband_rect)
subplot(3,1,2), stem(bin_vec_baseband_rcos)
subplot(3,1,3), stem(bin_vec_passband_ask)
```

![Figure](../lab5/lab5_figures/fig14.png)


* Observe the **Spectrum** of these signals using a *Signal Processing Tool* from Matlab. Type **sptool** in the command line and:
    * Go to **File -> Import** and select the **bin_vec_baseband_rect** signal. 
    * Click the top arrow to make this signal the imported Data.
    * Specify 4MHz as the Sampling frequency
    * Name the signal as **baseband_rect** and click **OK**.
    * Select this signal from the **Signal list** box in SPTool.
    * Click **Create** in the Spectra list.
    * Click **Apply** in the Spectrum Viewer.
    
    
![Figure](../lab5/lab5_figures/fig15.png)
   
   
* You should be able to see the spectrum of the rectangular data stream. Te baud rate is 125KHz, so this is supposed to be the bandwidth of this signal, as shown in the figure.


![Figure](../lab5/lab5_figures/fig16.png)


* **Repeat the previous steps** to be able to observe the spectrum of the baseband raised cosine data stream (*left plot*) and the passband ASK stream (*right plot*). Note that the pulse shapping process concentrates the pulse energy making the passband signal better fitted to be transmitted in passband communication channels.


![Figure](../lab5/lab5_figures/fig17.png)


* **Close** the Matlab and **Close** the *Hardware Manager* in Vivado.


* **Turn off** the board and **close** the Vivado project. 